# Reading Json File

In [90]:
from pathlib import Path
import re
import json

import pandas as pd
import xlwings as xw

In [91]:
base_path = Path(r'\\dkphysicspv1\e$\Gregs_Work\Temp\Plan Checking Temp')

file_name = 'JasonTestFile.txt'
test_file = base_path / file_name

output_tables = 'plan_info.xlsx'
output_file = base_path / output_tables

In [92]:
json_text = test_file.read_text(encoding='utf_8_sig')

**Need to clean `\` in user id**
> `JSONDecodeError: Invalid \escape: line 25 column 14 (char 575)`

From:<br>
`"UserId": "hc\gsalomon",`<br>
To:<br>
"UserId": "hc\\gsalomon",

In [93]:
json_text = json_text.replace('hc\\', 'hc\\\\')

**Remove extra text from field names**
```
"Fields": [{
  "FieldName": "• KV ANT •",
  "FieldId": "• KV ANT •",
```

In [94]:
# Remove extra marks from field names
json_text = re.sub(r'\s*•\s*', '', json_text)

**Need to deal with blank variables**

Some variables may return blanks.  Need to strip commas in those cases.

`JSONDecodeError: Expecting property name enclosed in double quotes: line 860 column 1 (char 23375)`

> `"HYPERARC": {`<br>
> `"IsHyperArcPlan": "",`<br>
> `},`<br>


In [95]:
 # Remove commas with no following object
json_text = re.sub(r',(\s*[}\]])', r'\1', json_text)


In [96]:
plan_data = json.loads(json_text)

In [97]:
list(plan_data.keys())

['PLAN SUM',
 'GENERAL',
 'PATIENT',
 'PROTOCOL',
 'DVH',
 'IMAGE',
 'STRUCTURE SET',
 'PLAN',
 'HYPERARC',
 'TARGET DOSE LEVEL',
 'DOSE',
 'REFERENCE POINT',
 'PLAN VARIATION',
 'EXTERNAL BEAM PLAN',
 'Calculation',
 'Fields']

In [98]:
wb = xw.Book()
wb.save(output_file)

In [99]:
structure_info = pd.DataFrame(plan_data["STRUCTURE SET"]["Structures"])
sheet = wb.sheets.add('Structures')
xw.view(structure_info, sheet)

In [100]:
structure_dose = pd.DataFrame(plan_data["DVH"]["DVHSummaryRows"])
sheet = wb.sheets.add('Structure Dose')
xw.view(structure_dose, sheet)

In [101]:
plan_info = plan_data['PLAN']
plan_info.update(plan_data['IMAGE'])
plan_info.update(plan_data['HYPERARC'])
plan_info.update(plan_data['EXTERNAL BEAM PLAN'])
sheet = wb.sheets.add('Plan Info')
xw.view(plan_info, sheet)

In [102]:
def get_fields(plan_data):
    for fld in plan_data['Fields']:
        if fld:
            yield fld['Field']

field_info = pd.DataFrame([fld for fld in get_fields(plan_data)]).T
sheet = wb.sheets.add('Field Info')
xw.view(field_info, sheet)

In [103]:
list(plan_data["Fields"][0].keys())

['Field',
 'Proton Field',
 'Wedge',
 'Bolus',
 'Block',
 'Tray',
 'Field Reference Point',
 'Carriage Group']

***Messages about calculation logs***
```
  Text=Calculation logs can be printed line by line using any of the following loops:
  Text= - CalculationWarnings
  Text= - CalculationErrors
  Text= - CalculationInfos
  Text= - CalculationNotes
  Text= - LmcCalculationErrors
  Text= - LmcCalculationWarnings
  Text= - LmcCalculationInfos
  Text= - LmcCalculationNotes
  Text= - CompCalculationErrors
  Text= - CompCalculationWarnings
  Text= - CompCalculationInfos
  Text= - CompCalculationNotes
  Text= - PortalDoseCalculationErrors
  Text= - PortalDoseCalculationWarnings
  Text= - PortalDoseCalculationInfos
  Text= - PortalDoseCalculationNotes
  Text= - IMRTOptimizationErrors
  Text= - IMRTOptimizationWarnings
  Text= - IMRTOptimizationInfos
  Text= - IMRTOptimizationNotes
  Text= - VMATOptimizationErrors
  Text= - VMATOptimizationWarnings
  Text= - VMATOptimizationInfos
  Text= - VMATOptimizationNotes
  Text= - BeamAngleOptimizationErrors
  Text= - BeamAngleOptimizationWarnings
  Text= - BeamAngleOptimizationInfos
  Text= - BeamAngleOptimizationNotes
  Text= - BLCalculationErrors
  Text= - BLCalculationWarnings
  Text= - BLCalculationInfos
  Text= - BLCalculationNotes
  Text= - BeamLineModifiers
  Text= - BLOptimizerCalculationErrors
  Text= - BLOptimizerCalculationWarnings
  Text= - BLOptimizerCalculationInfos
  Text= - BLOptimizerCalculationNotes
  Text= - PostProcessingCalculationErrors
  Text= - PostProcessingCalculationWarnings
  Text= - PostProcessingCalculationInfos
  Text= - PostProcessingCalculationNotes
  Text=Inside the loop the variable
  Text=  $$ LogLine
  Text=prints out the current line.
  Text=The entire dose calculation log for the active field is printed using:
  Text=$$ FieldLog :
  Text=$FieldLog
  ```